In [1]:
# Importing necessary lebraries
import numpy as np
import pandas as pd
import os
import sys


In [2]:
sys.path.append(os.path.join(os.path.abspath('..')))
# Import modules
from src import data_loading as dl

In [3]:

fraud_df = dl.load_data("Fraud_Data.csv")
credit_df = dl.load_data("creditcard.csv")

In [ ]:

sys.path.append(os.path.abspath("../"))

# Now import your scripts
from scripts import univariate as uni
from scripts import bivariate as bi
from scripts.FeatureEngineering import FeatureEngineering as fe  
from scripts.logger import logger 